# Read Data From CSV

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from numpy.linalg import norm
from quaternion import Quaternion
from scipy import stats
from scipy import signal
import matplotlib.pyplot as plt
import os
from os import walk
import madgwickahrs as mg
from quaternion import Quaternion


In [ ]:
#функции калибровки
def find_offsets(mpu,counts = 15):
#     calibration_set = mpu.loc[:counts]
    calibration_set =mpu 
    n = float(len(calibration_set))
    summ = []
    offsets = []
    for i in calibration_set:
        summ.append(calibration_set[i].sum())
    for s in summ:
        offsets.append(s/n)
    return offsets

def find_headers(mpu):
    headers = []
    for i in mpu:
        headers.append(i)
    return headers

def calibration(mpu,dataset):
    
    headers = find_headers(mpu)
#     print(headers)
    
    off = find_offsets(mpu)
    calibrated = dataset.copy()
    #не калибровать ускорение для фильтра маджвика!
#     calibrated[headers[0]] = mpu[headers[0]] - off[0]
#    calibrated[headers[1]] = mpu[headers[1]] - off[1]
#     calibrated[headers[2]] = mpu[headers[2]] - off[2]
    
    calibrated[headers[3]] = mpu[headers[3]] - off[3]
    calibrated[headers[4]] = mpu[headers[4]] - off[4]
    calibrated[headers[5]] = mpu[headers[5]] - off[5]
    
    return calibrated


In [ ]:
#функции для вывода графиков
def show_plots(mpu_t,mpu_b,title_t,title_b,accel=None):
    
    headers_t = find_headers(mpu_t)
    headers_b = find_headers(mpu_b)
    
    if accel == True:
        headers_t = headers_t[:3]
        headers_b = headers_b[:3]
        title_t += ' accel'
        title_b += ' accel'
    elif accel == None:
        headers_t = headers_t[:3]
        title_t +='accel'
        headers_b = headers_b[3:]
        title_b += ' gyro'
    else:
        headers_t = headers_t[3:]
        headers_b = headers_b[3:]
        title_t += ' gyro'
        title_b += ' gyro'
        


    fig = make_subplots(rows=1, cols=2, subplot_titles=(title_t,title_b))


    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[0]],name=headers_t[0]),1,1)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[1]],name=headers_t[1]),1,1)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[2]],name=headers_t[2]),1,1)

    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_b[headers_b[0]],name=headers_b[0]),1,2)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_b[headers_b[1]],name=headers_b[1]),1,2)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_b[headers_b[2]],name=headers_b[2]),1,2)


    fig.update_layout(legend_orientation="h",
                      legend=dict(x=.5, xanchor="center"),
                      hovermode="x",
                      margin=dict(l=0, r=0, t=40, b=0))
    fig.show()


def show_plot(mpu_t,title_t,accel=None):
    
    headers_t = find_headers(mpu_t)
    
    if accel == True:
        headers_t = headers_t[:3]
        title_t += ' accel'
    elif accel == None:
        headers_t = headers_t
    else:
        headers_t = headers_t[3:]
        title_t += ' gyro'
        


    fig = go.Figure() 
    fig.update_layout(title=title_t)
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[0]],name=headers_t[0]))
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[1]],name=headers_t[1]))
    fig.add_trace(go.Scatter(x=mpu_t.index, y=mpu_t[headers_t[2]],name=headers_t[2]))

    fig.show()


In [ ]:
def read_csv_file(file_path):
    df = pd.read_csv(file_path,sep = ' ',header = None)
    df.columns = ["s1","bt_ax","bt_ay","bt_az","bt_gx","bt_gy","bt_gz",
                 "s2","bb_ax","bb_ay","bb_az","bb_gx","bb_gy","bb_gz",
                 "s3","lt_ax","lt_ay","lt_az","lt_gx","lt_gy","lt_gz",
                 "s4","lb_ax","lb_ay","lb_az","lb_gx","lb_gy","lb_gz",
                "s5","rt_ax","rt_ay","rt_az","rt_gx","rt_gy","rt_gz",
                 "s6","rb_ax","rb_ay","rb_az","rb_gx","rb_gy","rb_gz",
                 ]

    return df

In [ ]:
#each  IMU for calibration
def parse_to_sensors(df):
    bt = df[["bt_ax","bt_ay","bt_az","bt_gx","bt_gy","bt_gz"]]
    bb = df[["bb_ax","bb_ay","bb_az","bb_gx","bb_gy","bb_gz"]]
    lt = df[["lt_ax","lt_ay","lt_az","lt_gx","lt_gy","lt_gz"]]
    lb = df[["lb_ax","lb_ay","lb_az","lb_gx","lb_gy","lb_gz"]]
    rt = df[["rt_ax","rt_ay","rt_az","rt_gx","rt_gy","rt_gz"]]
    rb = df[["rb_ax","rb_ay","rb_az","rb_gx","rb_gy","rb_gz"]]
    
#     show_plot(lb,'Non calibrated left-bot ',True)
#     show_plot(lb,'Non calibrated left-bot ',False)
    return bt,bb,lt,lb,rt,rb


# Filter outliers

In [ ]:
def filter_outliers(df_local):
    
    outl = find_headers(df_local)
    df= df_local.copy()
    for o in outl:
        median = df[o].median()
        std = df[o].std()
        df.loc[(df[o] - median).abs() > std,o]=np.nan
        df[o].fillna(median, inplace=True)
    return df

def find_outliers(df):
    
    bt,bb,lt,lb,rt,rb = parse_to_sensors(df)
    
    rb_outl = filter_outliers(rb)
    rt_outl =filter_outliers(rt)
    lb_outl = filter_outliers(lb)
    lt_outl = filter_outliers(lt)
    bb_outl = filter_outliers(bb)
    bt_outl = filter_outliers(bt)
    

    return bt_outl,bb_outl,lt_outl,lb_outl,rt_outl,rb_outl 


 Calibrate DF of each sensor

# Madgwick filter

In [ ]:
def find_orientation(calibrated_mpu):
    
    headers = find_headers(calibrated_mpu)
        
    quaternions = []

    heading = mg.MadgwickAHRS(sampleperiod = 0.06,beta = 1)
    for i,row in calibrated_mpu.iterrows():
        gyro_d  = []
        accel_d = []
        gyro_d.append(row[headers[3]])
        gyro_d.append(row[headers[4]])
        gyro_d.append(row[headers[5]])
        accel_d.append(row[headers[0]])
        accel_d.append(row[headers[1]])
        accel_d.append(row[headers[2]])

        heading.update_imu(gyro_d,accel_d)
        quaternions.append(heading.quaternion._get_q())

    # print(quaternions)
    pitch = []
    roll = []
    yaw = []
    for i in range(len(quaternions)):
        pitch.append(np.arcsin(2 * quaternions[i][1] * quaternions[i][2] + 2 * quaternions[i][0] * quaternions[i][3]))
        if np.abs(quaternions[i][1] * quaternions[i][2] + quaternions[i][3] * quaternions[i][0] - 0.5) < 1e-8:
            roll.append(0) 
            yaw.append(2 * np.arctan2(quaternions[i][1], quaternions[i][0]))
        elif np.abs(quaternions[i][1] * quaternions[i][2] + quaternions[i][3] * quaternions[i][0] + 0.5) < 1e-8:
            roll.append(-2 * np.arctan2(quaternions[i][1], quaternions[i][0]))
            yaw.append(0)
        else:
            roll.append(np.arctan2(2 * quaternions[i][0] * quaternions[i][1] - 2 * quaternions[i][2] *quaternions[i][3], 1 - 2 * quaternions[i][1] ** 2 - 2 *quaternions[i][3] ** 2))
            yaw.append(np.arctan2(2 * quaternions[i][0] * quaternions[i][2] - 2 * quaternions[i][1] * quaternions[i][3], 1 - 2 * quaternions[i][2] ** 2 - 2 * quaternions[i][3] ** 2))


    orientation_d = pd.DataFrame({'roll':np.rad2deg(roll),'pitch':np.rad2deg(pitch),'yaw':np.rad2deg(yaw)})
    return orientation_d

    

# Calibration data

In [ ]:

def all_calibrate(calibration_set,dataset):


#     bt_outl,bb_outl,lt_outl,lb_outl,rt_outl,rb_outl =  find_outliers(calibration_set)
    
    bt_outl,bb_outl,lt_outl,lb_outl,rt_outl,rb_outl =  find_outliers(calibration_set)
    bt_1,bb_1,lt_1,lb_1,rt_1,rb_1=parse_to_sensors(dataset)

    show_plot(lb_1,'first ',False)
    
    calibrated_bt = calibration(bt_outl,bt_1)
    calibrated_bb = calibration(bb_outl,bb_1)
    calibrated_lt = calibration(lt_outl,lt_1)
    calibrated_lb = calibration(lb_outl,lb_1)
    calibrated_rb = calibration(rb_outl,rb_1)
    calibrated_rt = calibration(rt_outl,rt_1)
   
       
    show_plot(calibrated_lb,'calibrated ',False)

#     print(calibrated_lb)
    orientation_bt = find_orientation(calibrated_bt)
    orientation_bb = find_orientation(calibrated_bb)
    orientation_lt = find_orientation(calibrated_lt)
    orientation_lb = find_orientation(calibrated_lb)
    orientation_rt = find_orientation(calibrated_rt)
    orientation_rb = find_orientation(calibrated_rb)

    
    return   orientation_bt, orientation_bb, orientation_lt, orientation_lb, orientation_rt, orientation_rb
    


In [ ]:
name = 'Vadim'
сalibration_set= read_csv_file(f'calibration/10_{name}.csv')


filenames = next(walk(f'D:/Thesis/{name}/'), (None, None, []))[2]  # [] if no file
filenames[0]
for i in range(len(filenames)):
    filename = f"D:/Thesis/{name}/{filenames[i]}"
    print(filename)
    IMU_set = read_csv_file(filename)
    orientation_bt, orientation_bb, orientation_lt, orientation_lb, orientation_rt, orientation_rb = all_calibrate(сalibration_set,IMU_set)
    ready_dataset = pd.concat([orientation_bt, orientation_bb, orientation_lt, orientation_lb, orientation_rt, orientation_rb],axis = 1)    
#     print(ready_dataset)
    ready_dataset.to_csv(f"D:\Thesis\dataset\lifting_orientation_{i}_0.csv", sep=",")




In [ ]:
test_plot = pd.read_csv('D:/Thesis/dataset/lifting_orientation_1_0.csv')

def preprocessing_dataset(dataset):
    df = dataset.drop(columns = "Unnamed: 0")
    df.columns = ["roll_bt","pitch_bt","yaw_bt",
                  "roll_bb","pitch_bb","yaw_bb",
                  "roll_lt","pitch_lt","yaw_lt",
                  "roll_lb","pitch_lb","yaw_lb",
                  "roll_rt","pitch_rt","yaw_rt",
                  "roll_rb","pitch_rb","yaw_rb"
                 ]
    return df
test_plot_1 = preprocessing_dataset(test_plot)

orientation_lb_test= test_plot_1[["roll_lb","pitch_lb","yaw_lb"]]
show_plot(orientation_lb_test,'Orientation left_bot sensor')


# plt.figure
# plt.plot(orientation_lb_test.index, orientation_lb_test["roll_lb"], 'b', alpha=0.75)
# plt.plot(orientation_lb_test.index, frame['pitch'], 'r', orientation_lb_test.index,  orientation_lb_test["yaw_lb"], 'g')
# plt.legend(('roll', 'pitch', 'yaw'), loc='best')
# plt.grid(True)
# plt.show()


In [ ]:
# orientation_lb_test.loc[:]

# Filter dataset

In [ ]:
def find_headers_orient(mpu):
    headers = []
    for i in mpu:
        headers.append(i)
    return headers

In [ ]:
def filter_scipy(dataset):
    xn = dataset
    b, a = signal.butter(3, 0.05)
    zi = signal.lfilter_zi(b, a)
    z, _ = signal.lfilter(b, a, xn, zi=zi* xn[0])
    z2, _ = signal.lfilter(b, a, z, zi=zi*z[0])
    y = signal.filtfilt(b, a, xn)
    plt.figure
    plt.plot(orientation_lb_test.index, xn, 'b', alpha=0.75)
    plt.plot(orientation_lb_test.index, z, 'r--', orientation_lb_test.index, z2, 'r', orientation_lb_test.index, y, 'k')
    plt.legend(('noisy signal', 'lfilter, once', 'lfilter, twice',
            'filtfilt'), loc='best')
    plt.grid(True)
    plt.show()
    
    return y

In [ ]:
# xn = orientation_lb_test['yaw_lb']
# b, a = signal.butter(3, 0.05)
# zi = signal.lfilter_zi(b, a)
# z, _ = signal.lfilter(b, a, orientation_lb_test['yaw_lb'], zi=zi* orientation_lb_test['yaw_lb'][0])

In [ ]:
# z2, _ = signal.lfilter(b, a, z, zi=zi*z[0])
# y = signal.filtfilt(b, a, xn)
headers_orient  = find_headers_orient(orientation_lb_test)
headers_orient

In [ ]:
filtered_roll =  filter_scipy(orientation_lb_test[headers_orient[0]])
filtered_pitch =  filter_scipy(orientation_lb_test[headers_orient[1]]) 
filtered_yaw =  filter_scipy(orientation_lb_test[headers_orient[2]])




In [ ]:

filtered_roll_pd = pd.DataFrame(filtered_roll)
filtered_pitch_pd = pd.DataFrame(filtered_pitch)
filtered_yaw_pd = pd.DataFrame(filtered_yaw)
frame = pd.concat([filtered_roll_pd,filtered_pitch_pd,filtered_yaw_pd],axis = 1)
frame.columns = ["roll", "pitch", "yaw"]
frame

In [ ]:
plt.figure
plt.plot(frame.index, frame['roll'], 'b', alpha=0.75)
plt.plot(frame.index, frame['pitch'], 'r', frame.index,  frame['yaw'], 'g')
plt.legend(('roll', 'pitch', 'yaw'), loc='best')
plt.grid(True)
plt.show()




# Orientation only with gyro 

In [ ]:


pitch = np.zeros(len(calibrated_lb))
yaw = np.zeros(len(calibrated_lb))
roll = np.zeros(len(calibrated_lb))

for i in range(len(calibrated_lb)-1):
    delta_t = 0.06
    
    pitch[i+1] = pitch[i] + calibrated_lb['lb_gy'][i]*delta_t
    yaw[i+1] = yaw[i]+calibrated_lb['lb_gz'][i]*delta_t
    roll[i+1] = roll[i]+calibrated_lb['lb_gx'][i]*delta_t
    

orientation_d = pd.DataFrame({'roll':np.rad2deg(roll),'pitch':np.rad2deg(pitch),'yaw':np.rad2deg(yaw)})

show_plot(orientation_d,'Orientation only with  gyro')

In [ ]:
#(-0.02213089742574257, 0.01005698124752475, -0.013309934386138617)